In [9]:
import sys
!{sys.executable} -m pip install trimesh
import trimesh
import inspect
import numpy as np
from collections import deque
from google.colab import files
from skimage.measure import marching_cubes

In [10]:
def load_dat(file_path):
    function_name = inspect.currentframe().f_code.co_name

    print(f"{function_name}: Load DAT from '{file_path}'")
    try:
        with open(file_path, 'rb') as f:
            binary_data = f.read()

        data_chunk = np.frombuffer(binary_data, dtype=np.uint8)
        print(f"{function_name}: DataChunk Size = {data_chunk.size}")
        return data_chunk

    except FileNotFoundError:
        print(f"{function_name}: Error: DAT '{file_path}' Not Found")
        exit()

    except Exception as e:
        print(f"{function_name}: Error: Unknown")
        print(e.__class__.__name__)
        print(e.args)
        print(e)
        print(f"{e.__class__.__name__}: {e}")
        exit()

In [11]:
def convert_data_chunk_into_3d_coordinate(data_chunk, x_dim, y_dim, z_dim):
    function_name = inspect.currentframe().f_code.co_name

    voxel_coords = []

    for i, value in enumerate(data_chunk):
        if value == 1:
            x = i % x_dim
            z = (i // x_dim) % z_dim
            y = (i // x_dim) // z_dim

            voxel_coords.append((x, y, z))

    return voxel_coords

In [12]:
def denoise(data_chunk, x_dim, y_dim, z_dim):
    function_name = inspect.currentframe().f_code.co_name

    voxel_coords = convert_data_chunk_into_3d_coordinate(data_chunk, x_dim, y_dim, z_dim)
    print(f"{function_name}: Filled Voxels = {len(voxel_coords)}")

    voxel_set = set(voxel_coords)
    visited = set()
    largest_component = []

    neighbor_offsets = []
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            for dz in [-1, 0, 1]:
                if (dx, dy, dz) != (0, 0, 0):
                    neighbor_offsets.append((dx, dy, dz))

    for start_voxel in voxel_coords:
        if start_voxel not in visited:
            current_component = []
            q = deque()

            q.append(start_voxel)
            visited.add(start_voxel)

            while q:
                cx, cy, cz = q.popleft()
                current_component.append((cx, cy, cz))

                for dx, dy, dz in neighbor_offsets:
                    nx, ny, nz = cx + dx, cy + dy, cz + dz

                    if not (0 <= nx < x_dim and \
                            0 <= ny < y_dim and \
                            0 <= nz < z_dim):
                        continue

                    neighbor_voxel = (nx, ny, nz)
                    if neighbor_voxel in voxel_set and neighbor_voxel not in visited:
                        visited.add(neighbor_voxel)
                        q.append(neighbor_voxel)

            if len(current_component) > len(largest_component):
                largest_component = current_component

    print(f"{function_name}: Denoised Voxels = {len(largest_component)}")

    result_binary_array = np.zeros(x_dim * y_dim * z_dim, dtype=np.uint8)
    for x, y, z in largest_component:
        index = x + z * x_dim + y * x_dim * z_dim

        if 0 <= index < result_binary_array.size:
            result_binary_array[index] = 1
        else:
            print(f"{function_name}: Worning: Invalid index {index} ({x}, {y}, {z}) was Skipped")

    return result_binary_array

In [ ]:
def perform_marching_cubes(data_chunk, x_dim, y_dim, z_dim, voxel_size):
    function_name = inspect.currentframe().f_code.co_name

    vertices = []
    faces = []

    try:
        original_volume = data_chunk.reshape((y_dim, z_dim, x_dim))
    except ValueError as e:
        print(f"{function_name}: Error: Reshape Failed {e}")
        print(f"Reshaped Form: ({y_dim}, {z_dim}, {x_dim})")
        print(f"DataChunk Size = {data_chunk.size}")
        exit()

    print(f"{function_name}: Apply Marching Cubes algorithm")
    verts, faces, normals, values = marching_cubes(original_volume, level=0.9)

    verts_scaled = verts * voxel_size

    faces = faces + 1

    vertices = verts_scaled.tolist()
    print(f"{function_name}: Vertex = {len(vertices)}")
    faces = faces.tolist()
    print(f"{function_name}: Faces = {len(faces)}")

    return vertices, faces, normals

In [ ]:
def transform_and_save_mesh(vertices, faces, normals, x_dim, y_dim, z_dim, voxel_size, file_path):
    function_name = inspect.currentframe().f_code.co_name

    center_x = (x_dim / 2.0 + 1) * voxel_size
    center_y = (y_dim / 2.0 + 1) * voxel_size
    center_z = (z_dim / 2.0 + 1) * voxel_size

    translation_vector = np.array([-center_x, -center_y, -center_z])

    transformed_vertices = []
    for v in vertices:
        v_np = np.array(v)

        translated_v = v_np + translation_vector

        transformed_vertices.append(translated_v.tolist())

    transformed_normals = []
    for n in normals:
        n_np = np.array(n)
        transformed_normals.append((n_np / np.linalg.norm(n_np)).tolist())

    print(f"{function_name}: Write OBJ to '{file_path}'")
    with open(file_path, 'w') as f:
        for v in transformed_vertices:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")

        for n in transformed_normals:
            f.write(f"vn {n[0]} {n[1]} {n[2]}\n")

        for face in faces:
            f.write("f")
            for v_idx in face:
                f.write(f" {v_idx}//{v_idx}")
            f.write("\n")

    print(f"{function_name}: Saved OBJ to '{file_path}'")

In [15]:
def obj2stl(obj_file_path, stl_file_path):
    function_name = inspect.currentframe().f_code.co_name

    print(f"{function_name}: Load OBJ from '{obj_file_path}'")
    try:
        mesh = trimesh.load(obj_file_path)
        rotation_matrix = trimesh.transformations.rotation_matrix(np.deg2rad(90), [1, 0, 0])
        mesh.apply_transform(rotation_matrix)

        print(f"{function_name}: Export STL to '{stl_file_path}'")
        mesh.export(stl_file_path)
        print(f"{function_name}: Saved STL to '{stl_file_path}'")

    except FileNotFoundError:
        print(f"{function_name}: Error: OBJ '{obj_file_path}' Not Found")
        exit()

    except Exception as e:
        print(f"{function_name}: Error: Unknown")
        print(e.__class__.__name__)
        print(e.args)
        print(e)
        print(f"{e.__class__.__name__}: {e}")
        exit()

In [16]:
input_file = "/content/24model.dat"
x_dim = 150
y_dim = 250
z_dim = 120
voxel_size = 0.002
output_obj_file = "output.obj"
output_stl_file = "output.stl"

data_chunk = load_dat(input_file)

denoised_data_chunk = denoise(data_chunk, x_dim, y_dim, z_dim)

vertices, faces, normals = perform_marching_cubes(denoised_data_chunk, x_dim, y_dim, z_dim, voxel_size)
# vertices, faces, normals = perform_marching_cubes(data_chunk, x_dim,y_dim, z_dim, voxel_size)

transform_and_save_mesh(vertices, faces, normals, x_dim, y_dim, z_dim, voxel_size, output_obj_file)

# obj2stl(output_obj_file, output_stl_file)

files.download(output_obj_file)

load_dat: Load DAT from '/content/24model.dat'
load_dat: DataChunk Size = 4500000
denoise: Filled Voxels = 931598
denoise: Denoised Voxels = 931443
perform_marching_cubes: Apply Marching Cubes algorithm
perform_marching_cubes: Vertex = 115810
perform_marching_cubes: Faces = 231640
transform_and_save_mesh: Write OBJ to 'output.obj'
transform_and_save_mesh: Saved OBJ to 'output.obj'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>